In [1]:
##Libraries##

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import insert
import pyodbc
import numpy as np
import datetime
import bccr
from bccr import PW
from bccr import SW
import lxml
from bs4 import BeautifulSoup

def ContraValor (mont,precio):
    result = 0
    
    sidechoice = input("Which ccy do you have: 'crc or usd?")
    
    if sidechoice == 'usd':
        result = mont * precio
    elif sidechoice == 'crc':
        result = mont / precio
    else: 
        print("you have to chose a valid ccy") 
    return result    


In [2]:
##Connection to cmv_uat database##

server = 'tcp:mycmvserver.database.windows.net' 
database = 'cmvdb_uat' 
username = 'ecervantes@cmvlatam.com' 
password = 'Columbia87/' 


connection = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};Server=tcp:mycmvserver.database.windows.net,1433;Database=cmvdb_uat;Uid='+username+';Pwd='+password+';Encrypt=yes;TrustServerCertificate=no;Connection Timeout=30;Authentication=ActiveDirectoryInteractive')
cursor = connection.cursor()

In [30]:
##Market Data##
##Static Market Data##
#monexpxlast = '619.61'       ##last traded price in Monex--
#monexpp = '619.76'           ##last update weighted average Monex--
##Live Market Data##
monexpxlast = float(1*SW(3435).iloc[-1])
monexpp = float(1*SW(3439).iloc[-1])

In [32]:
##internal price split--
px_bid_cover = '620.00'       
px_ask_cover = '621.00' 
ventbid = '616.00'
ventask = '625.00'

In [29]:
### Trade Form ### ### play to fill details ###
accionraw = input("Add new trade =1 or log lost trade =4?")
accion = int(accionraw)
print0 ="newtrade=1 / losttrade=4:  {:,} "
print(print0.format(accion))

sidechoiceraw,  = input("Which side bid=1 or ask=2?")
sidechoice = int(sidechoiceraw)
print1 ="Bid or Ask:  {:,} "
print(print1.format(sidechoice))


pxraw =  input ("precio a cliente?")
px = float(pxraw) 
print2 ="Precio a Cliente:  {:,} "
print(print2.format(px))

monedatengoraw = input ("USD=1 CRC=2 EUR=5")
monedatengo = int(monedatengoraw)   
print3 ="moneda input:  {:,} "
print(print3.format(monedatengo))

montoraw = input ("digite el monto de la moneda que tiene")
monto = float(montoraw)
print4 ="monto moneda input :  {:,} "
print(print4.format(monto)) 

clienteraw = input ("numero de cliente")
cliente = int(clienteraw)
   
print5 ="N. de cliente:  {:,} "
print(print5.format(cliente))


newtrade=1 / losttrade=4:  1 


Bid or Ask:  2 


Precio a Cliente:  623.5 


moneda input:  2 


monto moneda input :  30,000,000.0 


N. de cliente:  114,633 


In [31]:
## Play to print ticket to DB ##
##Fix setup computations##
sideid = f"{sidechoice}"                            ## 1 = bid  /  2 = ask
clienteid = f"{cliente}"                      ##de 6 digitos nada mas-- 
monex_pxlast = f"{monexpxlast}"
monex_pp = f"{monexpp}"
px_sp = f"{px}"                               ##precio sport --
accionid = f"{accion}"                           ## Add New = 1 , Delete = 2 , reinstate = 3 -- 3 under dev ##
#tradedate = 'GETDATE()'                      ##date
#valuedate = 'GETDATE()'                      ##date 
estadoid = '1'                                ##aplicada, pendiente, anulada--
dealerid = '2'                                ##trader that inputs the transaction-- 
productotipoid = '1'                          ##spot o forward-- 
#solicitud = '400113'
deletetradeid = ''                            ##Trade ID del trade a borrar--

if sideid == '1' :
    comccyid = '1'                            ##Moneda a comprar USD=1 CRC=2 EUR=5--                              
    venccyid = '2'
    px_cover = px_bid_cover                            ##moneda a vender USD=1 CRC=2 EUR=5--
else : 
    comccyid = '2'                            ##Moneda a comprar USD=1 CRC=2 EUR=5--                              
    venccyid = '1'                            ##moneda a vender USD=1 CRC=2 EUR=5--
    px_cover = px_ask_cover

###El blotter no esta cambiando la moneda a la hora de hacer el print
if (sideid == '1' ) and (monedatengo == 1 ):  ##Es un Bid y Moneda input es USD=1##    
    montocom = monto
    montoven = px * monto
elif (sideid == '1') and (monedatengo == 2): ##Es un Bid y Moneda input es CRC=2##
    montocom = monto / px
    montoven = monto
elif (sideid == '2') and (monedatengo == 1):  ##Es un Ask y Moneda input es USD=1##
    montocom = px * monto
    montoven = monto
elif (sideid == '2') and (monedatengo == 2):  ##Es un Ask y Moneda input es CRC=2##
    montocom = monto
    montoven = monto / px    
else :
    print("ccy conbination not supported")    

montocom = f"{montocom}" ##monto a comprar, expresado en moneda de compra--
montoven = f"{montoven}" 


### Working ticket entry from Python--must link to variables on top ####
if accionid == 2 :
    SQLCommand = ('INSERT INTO [dbo].[fxblotter.t] ( [createdat], [accionid], [sideid], [clienteid], [comccyid], [montocom], [px_sp], [venccyid], [montoven],  [tradedate], [valuedate], [px_cover], [monex_pxlast], [monex_pp], [estadoid], [dealerid], [productotipoid], [solicitud], [ventanillabid], [ventanillaask], [deletetradeid]) VALUES (GETDATE(),'+accionid+','+sideid+','+clienteid+','+comccyid+','+montocom+','+px_sp+','+venccyid+','+montoven+',GETDATE(),GETDATE(),'+px_cover+','+monex_pxlast+','+monex_pp+','+estadoid+','+dealerid+','+productotipoid+', 0,'+ventbid+','+ventask+','+deletetradeid+');')
    cursor.execute(SQLCommand)
    connection.commit()
    print(f"deletion of Trade"+deletetradeid+"printed ok")

else :
    SQLCommand = ('INSERT INTO [dbo].[fxblotter.t] ( [createdat], [accionid], [sideid], [clienteid], [comccyid], [montocom], [px_sp], [venccyid], [montoven],  [tradedate], [valuedate], [px_cover], [monex_pxlast], [monex_pp], [estadoid], [dealerid], [productotipoid], [solicitud], [ventanillabid], [ventanillaask]) VALUES (GETDATE(),'+accionid+','+sideid+','+clienteid+','+comccyid+','+montocom+','+px_sp+','+venccyid+','+montoven+',GETDATE(),GETDATE(),'+px_cover+','+monex_pxlast+','+monex_pp+','+estadoid+','+dealerid+','+productotipoid+', 0,'+ventbid+','+ventask+');')
    cursor.execute(SQLCommand)
    connection.commit()
    print(f"Trade printed ok")

Trade printed ok


In [81]:
##Calculadora 
pxcv = 619.50 
montocv = 7500000
print(ContraValor(montocv, pxcv))

12106.537530266343


In [69]:
  ##Deletes existing trade## CUIDADO!!!!!!

tiquetid = '####'     
SQLCommand = ('DELETE FROM [dbo].[fxblotter.t] WHERE tiquetid='+tiquetid+';')
cursor.execute(SQLCommand)
connection.commit()
print(f"Trade "+tiquetid+" deleted from blotter ok")

Trade 4129 deleted from blotter ok


In [81]:
  ##Amend monto, precio y side of existing trade## CUIDADO!!!!!!
tiquetid = '####' 
montocom = '12408233' 
px_sp = '620.55'
montoven = '19995.54'   
sideid = '2'
if sideid == '1' :
    comccyid = '1'                            ##Moneda a comprar USD=1 CRC=2 EUR=5--                              
    venccyid = '2'
                               ##moneda a vender USD=1 CRC=2 EUR=5--
else : 
    comccyid = '2'                            ##Moneda a comprar USD=1 CRC=2 EUR=5--                              
    venccyid = '1'                            ##moneda a vender USD=1 CRC=2 EUR=5--
    

SQLCommand = ('UPDATE [dbo].[fxblotter.t] SET [montocom]='+montocom+' WHERE tiquetid='+tiquetid+';')
cursor.execute(SQLCommand)
connection.commit()
SQLCommand = ('UPDATE [dbo].[fxblotter.t] SET [px_sp]='+px_sp+' WHERE tiquetid='+tiquetid+';')
cursor.execute(SQLCommand)
connection.commit()
SQLCommand = ('UPDATE [dbo].[fxblotter.t] SET [montoven]='+montoven+' WHERE tiquetid='+tiquetid+';')
cursor.execute(SQLCommand)
connection.commit()
SQLCommand = ('UPDATE [dbo].[fxblotter.t] SET [comccyid]='+comccyid+' WHERE tiquetid='+tiquetid+';')
cursor.execute(SQLCommand)
connection.commit()
QLCommand = ('UPDATE [dbo].[fxblotter.t] SET [venccyid]='+venccyid+' WHERE tiquetid='+tiquetid+';')
cursor.execute(SQLCommand)
connection.commit()
print(f"Trade "+tiquetid+" amended in blotter ok")

Trade 4130 amended in blotter ok
